In [1]:
from os.path import expanduser
import sys
home = expanduser("~")
sys.path.insert(2, f"{home}/catkin_ws/src/neurocontroller_database/src")
from load_bci_results import get_data_experiments
from utilities import load_json, color_axes, save_json, has_attr, root_dir, get_users, append_data, show_pdf
from ROS import load_object_space_ros_non_dom, load_object_space_ros
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

Tabla de frentes de Pareto. Se utiliza para generar el orden de los frentes, sí se desea combinar dos o más generaciones y extraer los no domindados se utiliza una coma para separarlos, ejemplo: "optimized-1,10" o "optimized-2,4,5"

In [2]:
def generate_table():
    front_list = []
    id_pos = {}

    for i in range(60):
        i += 1
        name = f"optimized-2"
        front_list.append(name)
        id_pos[name] = len(front_list) - 1

    
    data_front = {"id_position":id_pos,"fronts_table":front_list}
    save_json(data_front, f"{root_dir}/fronts_table.txt")


# generate_table()

Espacio de los objetivos con soluciones no dominadas

In [3]:

def plot_object_space(ini=1, end=61):
    fig = go.Figure()
    graphs = []
    individuals = np.empty((0,3))
    ids_pareto = []
    for i in range(ini,end):    
        id_front = f"optimized-{i}"
        
        # se extraen los individuos
        inds = load_object_space_ros_non_dom(id_front, 0)
        individuals = np.vstack((individuals, inds))

        # id de los ind
        ids_pareto += [id_front]*inds.shape[0]
        # Mesh3d
        # Surface
        graphs.append(go.Mesh3d(name=id_front,z=inds[:,2], x=inds[:,0], y=inds[:,1], showscale=False))


    # fig = go.Figure(data=[go.Mesh3d(x=data["x"], y=data["y"], z=data["z"], color='lightpink', opacity=0.50)])
    fig = go.Figure(graphs)
    fig.show()

    data = pd.DataFrame({"id":ids_pareto, "x":individuals[:,0], "y":individuals[:,1], "z":individuals[:,2]})
    fig = px.scatter_3d(data, x="x", y="y", z="z",color="id")
    fig.update_traces(marker_size = 5)
    fig.show()



plot_object_space(ini=51,end=61)

Rutas en el espacio de los objetivos

In [16]:


def concatenate_data(obj_reg, data_obj, color ):
    if len(data_obj.shape) == 1:
        shape = 1
        x = [data_obj[0]]
        y = [data_obj[1]]
        z = [data_obj[2]]
    else:
        shape = data_obj.shape[0]
        x = data_obj[:,0]
        y = data_obj[:,1]
        z = data_obj[:,2]

    append_data(obj_reg, "x", x, "concatenate")
    append_data(obj_reg, "y", y, "concatenate")
    append_data(obj_reg, "z", z, "concatenate")
    append_data(obj_reg, "color", [color]*shape, "concatenate")
    

def plot_roadmap():

    data_front = get_data_experiments("individuo", is_obj_space=False) 
    all_data = {}

    # por cada usuario
    for user_name in np.unique(data_front["usuario"]):
        if not has_attr(all_data, user_name):
            all_data[user_name] = {}
            
        data_user = data_front.loc[data_front["usuario"] == user_name]

        print("++++++++", user_name)
        # individuos seleccionados
        data_sel_ind = load_json(f"{root_dir}/{user_name}/history/selected_individuals.txt")


        for type_exp in np.unique(data_user["type experiment"]):
            data_type_exp = data_user.loc[data_user["type experiment"] == type_exp]

            for storage_pos in np.unique(data_type_exp["id experiment"]):
                data_exp = data_type_exp.loc[data_type_exp["id experiment"] == storage_pos]

                is_plot = False
                fig = go.Figure()
                # para cada frente

                # frentes
                ids_front = np.unique(data_exp["id pareto"])
                ids_front = [int(id.split("-")[1]) for id in ids_front]
                ids_front.sort()
                ids_front = [f"optimized-{id}" for id in ids_front]
                for id_pareto in  ids_front:
                    print("--------------")
                    if not has_attr(data_sel_ind, storage_pos):
                        continue
                    if not has_attr(data_sel_ind[storage_pos], id_pareto):
                        continue
                    print(id_pareto, type_exp, storage_pos)

                    # datos de selección
                    sel_data = data_sel_ind[storage_pos][id_pareto]
     
                    data_obj = load_object_space_ros_non_dom(id_pareto, non_dominated=False)
                    data_reg = {}
                    
                    num_inds = np.array(sel_data["selected_individuals"]["num_individuals"])
                    print(num_inds.shape, num_inds)
                    data_ind = data_obj[num_inds]
                    
                    # individuos seleccionados
                    if has_attr(sel_data, "selected_individuals"):
                        concatenate_data(data_reg, data_ind, "blue")
                        
                 
                    # punto cercano al de referencia
                    if has_attr(sel_data, "close_individual"):
                        num_ref_close = sel_data["close_individual"]["num_ind"]
                        # se busca la pos de point_close
                        pos_close = num_inds == num_ref_close
                        pos_close = pos_close + 0
                        pos_close = np.argmax(pos_close)
                        
                        # # la ruta se regresa al punto cercano
                        # pos_ind_return = np.arange(len(num_inds)-1-pos_close) + pos_close + 1 
                        # data_return = data_obj[pos_ind_return]
                        # if data_return.shape[0] > 1: 
                        #     data_return = np.flip(data_return, 0)
                        # # concatenate_data(data_reg, data_return, "blue")
                        
                        # se agrega el individuo cercano
                        ref_close = data_obj[num_ref_close]
                        concatenate_data(data_reg, ref_close, "green")
                        
                        print(f"num_inds_selected:{num_inds}")
                        print(f"ref_close:{ref_close} num_ind:{num_ref_close} id_pareto:{sel_data['close_individual']['id_pareto_front']}")
                        print(" values:",np.sum(sel_data["close_individual"]["point"] == ref_close))

                    # punto de referencia
                    # if has_attr(sel_data, "reference_point"):
                    #     ref_point = np.array(sel_data["reference_point"]["point"])
                    #     print(f"reference_point:{ref_point} num_ind:{sel_data['reference_point']['num_ind']} id_pareto:{sel_data['reference_point']['id_pareto_front']}")
                    #     concatenate_data(data_reg, ref_point, "red")

 
                    is_plot = True
                    all_data[user_name][id_pareto] = data_reg
                    fig.add_trace(go.Scatter3d(x=data_reg["x"], y=data_reg["y"], z=data_reg["z"], mode='lines+markers', name=id_pareto, marker=dict(color=data_reg["color"])))
                
                if is_plot:
                    fig.show()
                    
    return all_data

        

data_fig = plot_roadmap()

-- test_pref_old
-- user1
-- user_test
-- test_pref_new
-- compare_pref_model
Num Columnas 9
[('id pareto', 'SIZE: 80'), ('num ind', 'SIZE: 80'), ('num exp', 'SIZE: 80'), ('usuario', 'SIZE: 80'), ('type experiment', 'SIZE: 80'), ('id experiment', 'SIZE: 80'), ('PBI SAM', 'SIZE: 80'), ('valence SAM', 'SIZE: 80'), ('arousal SAM', 'SIZE: 80')]
++++++++ compare_pref_model
--------------
--------------
optimized-17 traditional 0
(4,) [72 80 10 11]
num_inds_selected:[72 80 10 11]
ref_close:[0.56397737 0.74775128 0.19206997] num_ind:80 id_pareto:optimized-17
 values: 3
--------------
optimized-19 traditional 0
(4,) [11  0 12  3]
num_inds_selected:[11  0 12  3]
ref_close:[0.52553731 0.64526701 0.27681086] num_ind:12 id_pareto:optimized-19
 values: 3
--------------
optimized-24 traditional 0
(4,) [13  1  5 14]
num_inds_selected:[13  1  5 14]
ref_close:[0.45099731 0.80194848 0.27630202] num_ind:1 id_pareto:optimized-24
 values: 3
--------------
optimized-26 traditional 0
(4,) [ 1 15  6  8]
num_i

--------------
--------------
optimized-19 traditional 3
(4,) [ 4  6 13 10]
num_inds_selected:[ 4  6 13 10]
ref_close:[0.45818914 0.89996517 0.27285997] num_ind:10 id_pareto:optimized-19
 values: 3
--------------
optimized-24 traditional 3
(4,) [15  1 13 14]
num_inds_selected:[15  1 13 14]
ref_close:[0.45307523 0.94443228 0.27506206] num_ind:15 id_pareto:optimized-24
 values: 3
--------------
optimized-29 traditional 3
(4,) [16  4  8 20]
num_inds_selected:[16  4  8 20]
ref_close:[0.45542516 0.76299511 0.24421107] num_ind:8 id_pareto:optimized-29
 values: 3
--------------
optimized-34 traditional 3
(4,) [19 10  2 14]
num_inds_selected:[19 10  2 14]
ref_close:[0.45479792 0.75371176 0.24776032] num_ind:10 id_pareto:optimized-34
 values: 3
--------------
optimized-39 traditional 3
(4,) [ 6  9 25  3]
num_inds_selected:[ 6  9 25  3]
ref_close:[0.45399094 0.75195604 0.28383978] num_ind:25 id_pareto:optimized-39
 values: 3
--------------
optimized-44 traditional 3
(4,) [16 25 43 41]
num_inds_s

++++++++ test_pref_new
--------------
--------------
optimized-19 traditional 0
(4,) [ 4  6 13 10]
num_inds_selected:[ 4  6 13 10]
ref_close:[0.45818914 0.89996517 0.27285997] num_ind:10 id_pareto:optimized-19
 values: 3
--------------
optimized-24 traditional 0
(4,) [15  1 13 14]
num_inds_selected:[15  1 13 14]
ref_close:[0.45307523 0.94443228 0.27506206] num_ind:15 id_pareto:optimized-24
 values: 3


--------------
--------------
optimized-19 traditional 1
(4,) [ 4  6 13 10]
num_inds_selected:[ 4  6 13 10]
ref_close:[0.45818914 0.89996517 0.27285997] num_ind:10 id_pareto:optimized-19
 values: 3
--------------
optimized-24 traditional 1
(4,) [15  1 13 14]
num_inds_selected:[15  1 13 14]
ref_close:[0.45307523 0.94443228 0.27506206] num_ind:15 id_pareto:optimized-24
 values: 3
--------------
optimized-29 traditional 1
(4,) [16  4  8 20]
num_inds_selected:[16  4  8 20]
ref_close:[0.45542516 0.76299511 0.24421107] num_ind:8 id_pareto:optimized-29
 values: 3
--------------
optimized-34 traditional 1
(4,) [19 10  2 14]
num_inds_selected:[19 10  2 14]
ref_close:[0.45479792 0.75371176 0.24776032] num_ind:10 id_pareto:optimized-34
 values: 3
--------------
optimized-39 traditional 1
(4,) [ 6  9 25  3]
num_inds_selected:[ 6  9 25  3]
ref_close:[0.45399094 0.75195604 0.28383978] num_ind:25 id_pareto:optimized-39
 values: 3


++++++++ test_pref_old
--------------
--------------
optimized-19 traditional 0
(4,) [ 4  6 13 10]
num_inds_selected:[ 4  6 13 10]
ref_close:[0.45818914 0.89996517 0.27285997] num_ind:10 id_pareto:optimized-19
 values: 3
--------------
optimized-24 traditional 0
(4,) [15  1 13 14]
num_inds_selected:[15  1 13 14]
ref_close:[0.45307523 0.94443228 0.27506206] num_ind:15 id_pareto:optimized-24
 values: 3
--------------
optimized-29 traditional 0
(4,) [16  4  8 20]
num_inds_selected:[16  4  8 20]
ref_close:[0.45542516 0.76299511 0.24421107] num_ind:8 id_pareto:optimized-29
 values: 3
--------------
optimized-34 traditional 0
(4,) [19 10  2 14]
num_inds_selected:[19 10  2 14]
ref_close:[0.45479792 0.75371176 0.24776032] num_ind:10 id_pareto:optimized-34
 values: 3


...

++++++++ user1
--------------
--------------
optimized-13 sam 1
(3,) [51 76 56]
num_inds_selected:[51 76 56]
ref_close:[0.40903869 0.7045     0.06767805] num_ind:76 id_pareto:optimized-13
 values: 0
--------------
optimized-24 sam 1
(3,) [13  1  5]
num_inds_selected:[13  1  5]
ref_close:[0.45099731 0.80194848 0.27630202] num_ind:1 id_pareto:optimized-24
 values: 0
--------------
optimized-29 sam 1
(3,) [ 9 16 20]
num_inds_selected:[ 9 16 20]
ref_close:[0.45399094 0.75195604 0.28383978] num_ind:16 id_pareto:optimized-29
 values: 0


--------------
--------------
optimized-17 traditional 0
(3,) [80 74 11]
num_inds_selected:[80 74 11]
ref_close:[0.56397737 0.74775128 0.19206997] num_ind:80 id_pareto:optimized-17
 values: 0
--------------
optimized-22 traditional 0
(3,) [6 7 4]
num_inds_selected:[6 7 4]
ref_close:[0.52553731 0.64526701 0.27681086] num_ind:6 id_pareto:optimized-22
 values: 0
--------------
optimized-27 traditional 0
(3,) [ 3  9 10]
num_inds_selected:[ 3  9 10]
ref_close:[0.52553731 0.64526701 0.27681086] num_ind:10 id_pareto:optimized-27
 values: 0
--------------
optimized-32 traditional 0
(3,) [ 3  7 17]
num_inds_selected:[ 3  7 17]
ref_close:[0.48947093 0.73582755 0.30890696] num_ind:7 id_pareto:optimized-32
 values: 0


++++++++ user_test
--------------
--------------
optimized-16 sam 4
(4,) [67 83 81 86]
num_inds_selected:[67 83 81 86]
ref_close:[0.43489629 0.8175859  0.09751423] num_ind:67 id_pareto:optimized-16
 values: 0
--------------
optimized-27 sam 4
(4,) [ 2  4  8 15]
num_inds_selected:[ 2  4  8 15]
ref_close:[0.44922251 0.86228454 0.07770482] num_ind:2 id_pareto:optimized-27
 values: 0
--------------
optimized-32 sam 4
(4,) [14 22 18  4]
num_inds_selected:[14 22 18  4]
ref_close:[0.45487061 0.82847431 0.10796919] num_ind:22 id_pareto:optimized-32
 values: 0


--------------
--------------
optimized-17 traditional 0
(3,) [62  1  4]
num_inds_selected:[62  1  4]
ref_close:[0.46420625 0.83465404 0.0759211 ] num_ind:4 id_pareto:optimized-17
 values: 0
--------------
optimized-22 traditional 0
(3,) [0 8 1]
num_inds_selected:[0 8 1]
ref_close:[0.46420625 0.83465404 0.0759211 ] num_ind:8 id_pareto:optimized-22
 values: 0
--------------
optimized-27 traditional 0
(3,) [ 4 15  5]
num_inds_selected:[ 4 15  5]
ref_close:[0.45778971 0.79228967 0.14552519] num_ind:15 id_pareto:optimized-27
 values: 0


--------------
--------------
optimized-17 traditional 3
(4,) [62  1  4 94]
num_inds_selected:[62  1  4 94]
ref_close:[0.46420625 0.83465404 0.0759211 ] num_ind:4 id_pareto:optimized-17
 values: 0
--------------
optimized-22 traditional 3
(4,) [0 8 1 4]
num_inds_selected:[0 8 1 4]
ref_close:[0.46420625 0.83465404 0.0759211 ] num_ind:8 id_pareto:optimized-22
 values: 0
--------------
optimized-27 traditional 3
(4,) [16 15  4  8]
num_inds_selected:[16 15  4  8]
ref_close:[0.46293835 0.81647863 0.07472731] num_ind:16 id_pareto:optimized-27
 values: 0
--------------
optimized-32 traditional 3
(4,) [13  6 14 27]
num_inds_selected:[13  6 14 27]
ref_close:[0.46293835 0.81647863 0.07472731] num_ind:13 id_pareto:optimized-32
 values: 0
--------------
optimized-37 traditional 3
(4,) [ 0 11 32  3]
num_inds_selected:[ 0 11 32  3]
ref_close:[0.46355422 0.99777766 0.04019854] num_ind:32 id_pareto:optimized-37
 values: 0
--------------
optimized-42 traditional 3
(4,) [48  7 38 24]
num_inds_selected:[

In [ ]:
data_fig["user1"]



{'optimized-13': {'x': array([18.23845343, 17.76436863, 18.35595472, 17.76436863]),
  'y': array([0.73794912, 0.67387812, 0.73318187, 0.67387812]),
  'z': array([0.00816991, 0.16680994, 0.01110044, 0.16680994]),
  'color': array(['blue', 'blue', 'blue', 'green'], dtype='<U32')},
 'optimized-24': {'x': array([19.45559539, 19.36752754, 19.48190867, 19.36752754]),
  'y': array([0.75481556, 0.76572136, 0.71860449, 0.76572136]),
  'z': array([0.76717943, 0.68101212, 0.69959068, 0.68101212]),
  'color': array(['blue', 'blue', 'blue', 'green'], dtype='<U32')},
 'optimized-29': {'x': array([19.45559539, 19.48190867, 19.5127418 , 19.48190867]),
  'y': array([0.75481556, 0.71860449, 0.72025922, 0.71860449]),
  'z': array([0.76717943, 0.69959068, 0.61066447, 0.69959068]),
  'color': array(['blue', 'blue', 'blue', 'green'], dtype='<U32')},
 'optimized-17': {'x': array([23.68428085, 22.65075086, 22.21555944, 23.68428085]),
  'y': array([0.71464159, 0.75041479, 0.61805222, 0.71464159]),
  'z': array

In [ ]:
# data_fig

Calcula los rangos de los objetivos. Se utiliza para mapear el tamaño de circulo

In [ ]:
# {"min": [2.13577841, 0.00990099, 1.67e-06, -18.87535688], "max": [40.34387611, 0.95238095, 2.46473372, 0.02382005]}


def calculate_ranges(axis):
    ids_pareto = []
    individuals = np.empty((0,3))

    for i in range(60): 
        i+=1
        id_front = f"optimized-{i}"
        
        inds = load_object_space_ros(id_front)
        individuals = np.vstack((individuals, inds))

        # id de los ind
        ids_pareto += [id_front]*inds.shape[0]
    
    range_min = np.min(individuals, axis=0)
    range_max = np.max(individuals, axis=0)
    save_json({"min": range_min.tolist(), "max":range_max.tolist()}, f"{root_dir}/range_objectives.txt")
    # print(range_min, range_max)

    pd_d = pd.DataFrame({"id":ids_pareto, "x":individuals[:,0], "y":individuals[:,1], "z":individuals[:,2]})

    fig = px.scatter(pd_d, x=axis, color="id")
    fig.show()

    if axis == "x":
        pos = 0
    if axis == "y":
        pos = 1
    if axis == "z":
        pos = 2

    pd_d[axis] = np.divide(np.subtract(pd_d[axis], [range_min[pos]]), [range_max[pos]-range_min[pos]])

    fig = px.scatter(pd_d, x=axis, color=axis)
    fig.show()

calculate_ranges("z")